# Analyze customer reviews with Gemini using Python Notebooks

In this section of this notebook, you will use BigQuery, Gemini, and Python to determine sentiment for customer reviews.

You will need some Python libraries for this notebook.  Run the cell below to load the libraries.

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.cloud import storage

In [ ]:
project_id = "<enter your project id here>" # enter your project id
region = "us-central1"

You will need to create an external cloud resource connection to connect to the Gemini Pro model in Vertex AI. Note: This can be done here in this notebook with a cli command, but cannot be done with a SQL.  Alternatively you can create this using the Cloud Console.

In [ ]:
!bq mk --connection \
  --connection_type=CLOUD_RESOURCE \
  --location=US \
  gemini_conn

Note: To run the following cells without error, the resource connection needs to be successfully created, and permissions applied to the service account for it including access to cloud storage bucket as the storage object admin role, and the vertex AI user role.  To do this complete the following steps:



### Retrieve the service account id for the resource connection

1. In the explorer to the left of this notebook, expand **External connections**.

1. Click **us.gemini_conn**.

1. On the connection info page, copy the service account id for the connection.

### Grant IAM permissions to the connection's service account

Granting IAM permissions to the resource connection's service account before you start working in BigQuery will ensure you do not encounter access denied errors when running queries.

1. In the Google Cloud Console, on the **Navigation menu**, click **CloudStorage**.

1. Return to the root of the bucket.

1. Click **PERMISSIONS**.

1. Click **GRANT ACCESS**.

1. In the **New principals** field, enter the service account ID you copied earlier.

1. In the Select a role field, enter **Storage Object**, and then select **Storage Object Admin** role.

1. Click **Save**.

   The result is the service account id now includes the Storage Object Admin role.

### Task 2. Grant IAM permissions to the connection's service account

In this task, you grant the cloud resource connection's service account IAM permissions, through a role, to enable it access the Vertex AI services.

1. In the Google Cloud Console, on the **Navigation menu**, click **IAM & Admin**.

1. Click **Grant Access**.

1. In the **New principals** field, enter the service account ID that you copied earlier.

1. In the Select a role field, enter **Vertex AI**, and then select **Vertex AI User** role.

1. Click **Save**.

   The result is the service account id now includes the Vertext AI User role.

Now that the service account for the resource connection has the appropriate roles you can continue on to the next cell. You will need a BigQuery Dataset. The cell below creates a new BigQuery dataset in the US region and is named gemini_demo. Note: You will need to enter your project id in the second line of this cell for the code to work.

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `<enter your project id here>.gemini_demo`
OPTIONS(location="US");

Load a BigQuery table with the customer reviews data, as stored in a csv file on CloudStorage.  Note: You will need to enter your project id in the last line of this cell for the code to work.


In [ ]:
%%bigquery
LOAD DATA OVERWRITE gemini_demo.customer_reviews
(customer_review_id INT64, customer_id INT64, location_id INT64, review_datetime DATETIME, review_text STRING, social_media_source STRING, social_media_handle STRING)
FROM FILES (
  format = 'CSV',
  uris = ['gs://<enter your project id here>-bucket/gsp1249/customer_reviews.csv']);

Create the Gemini Pro model in BigQuery. The model will be used in the next step.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `gemini_demo.gemini_pro`
REMOTE WITH CONNECTION `us.gemini_conn`
OPTIONS (endpoint = 'gemini-pro')

The model will show up in the models section underneath the gemini_demo dataset. To access the model, expand the gemini_demo dataset in the Explorer pane to the left of this notebook. Then expand the models section and click on the gemini_pro model to review its details.

Create a BigQuery table with the results of sentiment analysis provided by the Gemini Pro model you just created for each customer review included in the gemini_demo.customer_reviews table.  Note: This step will take approximately 4 to 5 minutes to complete.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE
`gemini_demo.customer_reviews_analysis` AS (
SELECT ml_generate_text_llm_result, social_media_source, review_text, customer_id, location_id, review_datetime
FROM
ML.GENERATE_TEXT(
MODEL `gemini_demo.gemini_pro`,
(
   SELECT social_media_source, customer_id, location_id, review_text, review_datetime, CONCAT(
      'Classify the sentiment of the following text as positive or negative.',
      review_text, "In your response don't include the sentiment explanation. Remove all extraneous information from your response, it should be a boolean response either positive or negative.") AS prompt
   FROM `gemini_demo.customer_reviews`
),
STRUCT(
   0.2 AS temperature, TRUE AS flatten_json_output)));

Once the previous cell is complete, you can use SQL to review the records.

In [ ]:
%%bigquery
SELECT * FROM `gemini_demo.customer_reviews_analysis`
LIMIT 1000

Notice how some of the sentiment results in the ml_generate_text_llm_result column may not be formated as positive, mixed or negative.  For example you may see reults with **Positive.** Or they may include a result like this but include a trailing **\n** for a line break.  We can create view to correct results like these and store them in a BigQuery view.

Also, notice how some records may have Google Reviews or Google My Business as the social_media source for the review. The view below can be used to consolidate reviews into one source for Google.

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW gemini_demo.cleaned_data_view AS
SELECT REPLACE(REPLACE(LOWER(ml_generate_text_llm_result), '.', ''), ' ', '') AS sentiment,
REGEXP_REPLACE(
      REGEXP_REPLACE(
            REGEXP_REPLACE(social_media_source, r'Google(\+|\sReviews|\sLocal|\sMy\sBusiness|\sreviews|\sMaps)?', 'Google'),
            'YELP', 'Yelp'
      ),
      r'SocialMedia1?', 'Social Media'
   ) AS social_media_source,
review_text, customer_id, location_id, review_datetime
FROM `gemini_demo.customer_reviews_analysis`;

You can examine the view by using the query below.

In [ ]:
%%bigquery
SELECT * FROM `gemini_demo.cleaned_data_view`

Now that the sentiment analysis is complete, and sentiment column is properly formatted we can use it to create a report and a chart with the total count of positive versus negative customer reviews.

To accomplish this we will transfer the records from the BigQuery View to a Python dataframe, then we will observe the first 10 records in the dataframe with the cell below.

In [ ]:
client = bigquery.Client()
query = "SELECT sentiment, COUNT(*) AS count FROM `gemini_demo.cleaned_data_view` WHERE sentiment IN ('positive', 'negative') GROUP BY sentiment;"
query_job = client.query(query)
results = query_job.result().to_dataframe()
results.head(10)

The result is a table with total counts of positive and negative reviews.

With these counts we can use Python and the matplot lib library to generate a bar chart by running the two cells below.

In [ ]:
sentiment = results["sentiment"].tolist()
count = results["count"].tolist()

In [ ]:
plt.bar(sentiment, count, color='skyblue')
plt.xlabel("Sentiment")
plt.ylabel("Total Count")
plt.title("Bar Chart from BigQuery Data")
plt.show()

# Audio review sentiment analysis and customer service response

Data beans wants to experiment with customer reviews using audio recordings. In this section of this notebook you will use CloudStorage, BigQuery, Gemini Flash, and Python to perform sentiment analysis on customer reviews provided to data beans as audio files. And from the resulting analysis you will generate customer service responses to be sent back to the customer thanking them for their review.

## Review the audio files as stored in CloudStorage
The audio files are stored in a bucket within your account. Access CloudStorage now and find the bucket named with your project id-bucket. Open the gsp1249 folder. Open the audio subfolder. You will see 5 mp3 files, you can play files by accessing the public link.

Once you have reviewed the audio files, return here and run the cell below to perform the sentiment analysis and retrieve the customer service response for each review.

A few key points about this cell:

* The first line initializes Vertex AI with your project ID and region, you will need to populate these values.
* The next line creates a model in BigQuery named model, based upon the Gemini Flash model.
* You need to set your bucket as the bucket_name string variable. Note: folder_name is used as well for gsp1249/audio subfolder.  Don't change this.
* You then define a prompt to be used by the Gemini Flash model. The prompt effectively converts the audio file to text, then analyzes the sentiment of the text, and once the analysis is complete, creates a customer response for each file.
* A function called list_mp3_files is created idetify all mp3 files within the bucket.
* Then these files are processed by the Gemini Flash model within the if statement.



In [ ]:
vertexai.init(project=project_id, location=region)

model = GenerativeModel(model_name="gemini-1.5-flash")

prompt = """
Please provide a transcript for the audio.
Then provide a summary for the audio.
Be concise and short.
Do not make up any information that is not part of the audio and do not be verbose.
Then determine the sentiment of the audio: positive, neutral or negative.

Also, you are a customr service representative.
How would you respond to this customer review?
Answer in JSON format with four keys: transcript, summary, sentiment, and response. Transcript should be a string, summary should be a sting, sentiment should be a string, and customer response should be a string.
"""

bucket_name = project_id + '-bucket'
folder_name = 'gsp1249/audio'  # Include the trailing '/'

def list_mp3_files(bucket_name, folder_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    print('Accessing ', bucket, ' with ', storage_client)

    blobs = bucket.list_blobs(prefix=folder_name)

    mp3_files = []
    for blob in blobs:
        if blob.name.endswith('.mp3'):
            mp3_files.append(blob.name)
    return mp3_files

file_names = list_mp3_files(bucket_name, folder_name)
if file_names:
    print("MP3 files found:")
    print(file_names)
    for file_name in file_names:
        audio_file_uri = f"gs://{bucket_name}/{file_name}"
        print('Processing file at ', audio_file_uri)
        audio_file = Part.from_uri(audio_file_uri, mime_type="audio/mpeg")
        contents = [audio_file, prompt]
        response = model.generate_content(contents)
        print(response.text)
else:
    print("No MP3 files found in the specified folder.")